In [46]:
import pandas as pd
import os
import glob
from datetime import datetime as dt
import copy
import numpy as np
import plotly.express as px
myPath = os.getcwd() # current working directory - change this to the location of your exported google data

In [ ]:
def extractData(myDict,keys):
    """
    Reads a nested dictionary.
    
    Args:
        myDict (dict of dict of...): Nested dictionary of arbitrary number of nestings, e.g. like what you get in a json file.
        
        keys (list of strs): List of keys corresponding to the nested dictionaries you want to exract.
        
    Returns:
        element (object): myDict[key0][key1]...[keyn]
    
    Created on 21/03/2021 by Bennett Schneider
    
    """
    subDict = copy.deepcopy(myDict)

    try:
        for key in keys:
            subDict = subDict[key] # recursively pull out keys until you can't anymore
        element = subDict # return the bottom element
    except KeyError:
        element = np.nan # if the key doesn't exist, return none
    except TypeError:
        element = subDict # if you go too far and you've found a non-dict element, return this

    return element

In [2]:
# Rename the unzipped folder you download from google to json. 
# Inside, it should have Takeout/Location History/Semantic Location History
tPath = os.path.join(myPath,r'json','Takeout','Location History','Semantic Location History')

In [3]:
# Get list of json files representing your semantic location. Grouped by year. One file per month
myFiles = []
for subdir, dirs, files in os.walk(os.path.join(tPath)):
    for filename in files:
        myFiles.append(os.path.join(subdir,filename))

In [10]:
# Read all the data into a pandas data and concatenate vertically
myData = []
for file in myFiles:
    myData.append(pd.read_json(file))
myData = pd.concat(myData)

In [ ]:
# Pull placeVisit data out of the pandas dataframe

procData = myData.copy() # copy myData so we don't overwite it in the subsequent steps

# Pull data from the placeVisit -> location position
for col in ['name','address','latitudeE7','longitudeE7']:
    procData[col] = procData['timelineObjects'].map(lambda x: extractData(x,['placeVisit','location',col]))
    
procData = procData[~procData['name'].isna()] # remove rows with no name

# Convert E7 format to standard lat/lon
procData['lat'] = procData['latitudeE7'] / 10.**7
procData['lon'] = procData['longitudeE7'] / 10.**7

In [ ]:
# Pull out timestamps under the placeVisit -> duration position
for col in ['startTimestampMs','endTimestampMs']:
    procData[col] = procData['timelineObjects'].map(lambda x: extractData(x,['placeVisit','duration',col]))
    procData[col] = procData[col].astype(float)/1000 # convert str in ms to float in micro seconds
    procData[col.replace('TimestampMs','')] = procData[col].map(lambda x: dt.fromtimestamp(x)) # convert to datetime

In [76]:
# Flag the the data with total monthly duration at each place
procData['Month'] = procData['start'].map(lambda x: float(dt(x.year,x.month,1).strftime('%Y%m'))) # monthly flag as a float
procData['Duration'] = (procData['end'] - procData['start']) # duration at each location
procData['Duration'] = procData['Duration'].map(lambda x: x.total_seconds()) # convert duration from timedelta to seconds

groupedData = procData.copy() # copy procData so we don't overwrite it in the next step

# Aggregate so we get the total duration spent in ea
groupedData = groupedData.groupby(['Month','name','address']).agg(
    {
        'lat':np.mean,
        'lon':np.mean,
        'Duration':np.sum,
        'start':np.min,
        'end':np.max
    }).reset_index()

In [82]:
groupedData

,Month,name,address,lat,lon,Duration,start,end
0,201402.0,11 Nioka Ave,11 Nioka Ave\nMalua Bay NSW 2536\nAustralia,-35.789863,150.229552,157401.902,2014-02-05 12:49:37.589,2014-02-07 16:31:09.493
1,201402.0,14B O'Connell St,14B O'Connell St\nAinslie ACT 2602\nAustralia,-35.263227,149.146316,140034.373,2014-02-05 09:14:46.015,2014-02-27 14:39:24.866
2,201402.0,250 Baroona Rd,250 Baroona Rd\nMichelago NSW 2620\nAustralia,-35.782298,149.139411,1678128.285,2014-02-07 22:12:22.646,2014-03-03 15:06:59.751
3,201402.0,ANU Union,3 Rimmer St\nCanberra ACT 2601\nAustralia,-35.277082,149.120649,1142.014,2014-02-13 13:22:53.953,2014-02-13 13:41:55.967
4,201402.0,Batemans Bay Surf Life Saving Club,560 George Bass Dr\nMalua Bay NSW 2536\nAustralia,-35.793706,150.229734,4550.201,2014-02-05 14:00:47.494,2014-02-06 23:01:03.001
...,...,...,...,...,...,...,...,...
3524,202103.0,UTOPIA vegan cuisine,"Dickson Plaza, Shop 5/28 Challis Street\nWooll...",-35.249334,149.136419,3581.353,2021-03-01 20:10:25.745,2021-03-01 21:10:07.098
3525,202103.0,Uriarra East,Stromlo ACT 2611\nAustralia,-35.249532,148.953053,2066.751,2021-03-01 18:46:21.669,2021-03-01 19:20:48.420
3526,202103.0,Windlab Limited,60 Marcus Clarke St\nCanberra ACT 2601\nAustralia,-35.275861,149.127130,428754.632,2021-03-01 08:52:40.681,2021-03-19 19:32:23.549
3527,202103.0,Woolworths Dickson,1 Dickson Pl\nDickson ACT 2602\nAustralia,-35.249974,149.138625,965.073,2021-03-09 20:50:24.567,2021-03-09 21:06:29.640


In [87]:
# Plot all points on a map
fig = px.scatter_mapbox(
    groupedData,
    lat="lat",
    lon="lon",
    hover_data=["name", "address",'start','end'],
    color='Month',
    size='Duration',
    color_continuous_scale ='Viridis', zoom=3, height=1000,width=1800)
fig.update_layout(mapbox_style="open-street-map")
fig.write_html(os.path.join(myPath,'locationsHistory.html'))
fig.show()

In [49]:
procData

,timelineObjects,name,address,latitudeE7,startTimestampMs,endTimestampMs,longitudeE7,lat,lon,start,end
2,{'placeVisit': {'location': {'latitudeE7': 455...,Shakti Rock Gym,"175 R. St Viateur Est\nMontréal, QC H2T 1B4\nC...",455275553.0,1.396371e+09,1.396380e+09,-7.359797e+08,45.527555,-73.597967,2014-04-02 03:57:57.422,2014-04-02 06:12:44.072
9,{'placeVisit': {'location': {'latitudeE7': 455...,Centre Mont-Royal,"2200 Rue Mansfield\nMontréal, QC H3A 3R8\nCanada",455027242.0,1.396459e+09,1.396468e+09,-7.357469e+08,45.502724,-73.574686,2014-04-03 04:23:32.642,2014-04-03 06:44:38.060
16,{'placeVisit': {'location': {'latitudeE7': 455...,Tim Hortons,"666 Rue Sherbrooke Ouest\nMontréal, QC H3A 0B2...",455049562.0,1.396540e+09,1.396540e+09,-7.357319e+08,45.504956,-73.573194,2014-04-04 02:39:29.515,2014-04-04 02:50:28.059
18,{'placeVisit': {'location': {'latitudeE7': 454...,Alexis Nihon,"1500 Avenue Atwater\nMontréal, QC H3Z 1X5\nCanada",454890110.0,1.396541e+09,1.396542e+09,-7.358602e+08,45.489011,-73.586023,2014-04-04 03:01:35.382,2014-04-04 03:21:37.093
25,{'placeVisit': {'location': {'latitudeE7': 454...,Cinéma Cineplex Forum,2313 Rue Sainte-Catherine O Suite 101\nMontréa...,454897905.0,1.396547e+09,1.396560e+09,-7.358494e+08,45.489790,-73.584941,2014-04-04 04:36:57.672,2014-04-04 08:21:40.880
...,...,...,...,...,...,...,...,...,...,...,...
157,{'placeVisit': {'location': {'latitudeE7': -35...,Canberra Centre,148 Bunda St\nCanberra ACT 2601\nAustralia,-352802271.0,1.616053e+09,1.616064e+09,1.491333e+09,-35.280227,149.133256,2021-03-18 18:31:57.780,2021-03-18 21:36:55.291
159,{'placeVisit': {'location': {'latitudeE7': -35...,14B O'Connell St,14B O'Connell St\nAinslie ACT 2602\nAustralia,-352632304.0,1.616064e+09,1.616103e+09,1.491463e+09,-35.263230,149.146304,2021-03-18 21:47:14.318,2021-03-19 08:28:01.000
161,{'placeVisit': {'location': {'latitudeE7': -35...,Windlab Limited,60 Marcus Clarke St\nCanberra ACT 2601\nAustralia,-352758614.0,1.616103e+09,1.616143e+09,1.491271e+09,-35.275861,149.127130,2021-03-19 08:37:46.515,2021-03-19 19:32:23.549
163,{'placeVisit': {'location': {'latitudeE7': -35...,Ovolo Nishi,NewActon Precinct\n25 Edinburgh Ave\nCanberra ...,-352851301.0,1.616143e+09,1.616151e+09,1.491226e+09,-35.285130,149.122553,2021-03-19 19:36:25.655,2021-03-19 21:44:01.359


In [75]:
myData['']

,timelineObjects
0,"{'activitySegment': {'startLocation': {}, 'end..."
1,"{'activitySegment': {'startLocation': {}, 'end..."
2,{'placeVisit': {'location': {'latitudeE7': 455...
3,"{'activitySegment': {'startLocation': {}, 'end..."
4,"{'activitySegment': {'startLocation': {}, 'end..."
...,...
161,{'placeVisit': {'location': {'latitudeE7': -35...
162,{'activitySegment': {'startLocation': {'latitu...
163,{'placeVisit': {'location': {'latitudeE7': -35...
164,{'activitySegment': {'startLocation': {'latitu...


In [67]:
data['timestamp_ms'] = data['locations'].map(lambda x: x['timestampMs'])

# convert lat/lon to decimalized degrees and the timestamp to date-time
data['lat'] = data['lat'] / 10.**7
data['lon'] = data['lon'] / 10.**7
data['timestamp_ms'] = data['timestamp_ms'].astype(float) / 1000
data['datetime'] = data['timestamp_ms'].map(lambda x: dt.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
date_range = '{}-{}'.format(data['datetime'].min()[:4], data['datetime'].max()[:4])

<>:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
<ipython-input-67-da8d081251f6>:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
  'test'['test']


TypeError: string indices must be integers

In [52]:
myData

[]